In [1]:
from collections import Counter

def get_wordle_pattern(guess, answer):
    """
    Generate Wordle feedback pattern.
    Returns tuple: 0=gray (not in word), 1=yellow (wrong position), 2=green (correct)
    """
    result = [0] * 5
    answer_counts = Counter(answer)
    
    # First pass: mark all exact matches (green)
    for i, (g, a) in enumerate(zip(guess, answer)):
        if g == a:
            result[i] = 2
            answer_counts[g] -= 1  # Consume this letter
    
    # Second pass: mark letters in wrong positions (yellow)
    for i, g in enumerate(guess):
        if result[i] == 0 and answer_counts[g] > 0:
            result[i] = 1
            answer_counts[g] -= 1  # Consume this letter
    
    return tuple(result)

def apply_constraints(word, guess, pattern):
    """Check if a word is consistent with observed guess/pattern"""
    return get_wordle_pattern(guess, word) == pattern

# Load word list from NLTK
print("=== Loading Word List ===\n")

try:
    import nltk
    from nltk.corpus import words
    
    # Download if not already present
    try:
        words.words()
    except LookupError:
        print("Downloading NLTK words corpus...")
        nltk.download('words', quiet=True)
    
    # Get all 5-letter words, uppercase
    all_words = words.words()
    WORD_LIST = sorted(set(word.upper() for word in all_words if len(word) == 5 and word.isalpha()))
    
    print(f"Loaded {len(WORD_LIST)} five-letter words from NLTK")
    print(f"Sample words: {WORD_LIST[:10]}")
    print()
    
except ImportError:
    print("NLTK not installed. Using fallback word list.")
    # Fallback to a larger curated list
    WORD_LIST = ["SALET", "ROATE", "RAISE", "ARISE", "IRATE", "CRANE", "SLATE", 
                 "CRATE", "TRACE", "STARE", "ADORE", "ALONE", "ATONE", "STORE",
                 "SHORE", "SNORE", "SPORE", "SCORE", "SWORE", "SHONE", "STONE",
                 "DRONE", "PRONE", "OZONE", "PHONE", "THOSE", "WHOSE", "CHOSE",
                 "BRAKE", "FLAKE", "SHAKE", "SNAKE", "QUAKE", "AWAKE"]
    print(f"Using fallback list: {len(WORD_LIST)} words\n")

# Demonstration
print("=== Wordle Feedback Mechanism ===\n")

test_cases = [
    ("ROBOT", "FLOOR"),  # Repeated letters
    ("CRANE", "REACT"),  # Anagrams
    ("SALET", "LASER"),  # Close match
]

for guess, answer in test_cases:
    pattern = get_wordle_pattern(guess, answer)
    pattern_str = ''.join(['⬜' if p == 0 else '🟨' if p == 1 else '🟩' for p in pattern])
    print(f"Guess: {guess}")
    print(f"Answer: {answer}")
    print(f"Pattern: {pattern_str} {pattern}")
    print()

# Show constraint propagation with realistic word list
print("=== Constraint Propagation Example ===\n")
# Select words starting with 'FL' from full list
sample_words = [w for w in WORD_LIST if w.startswith('FL')][:8]
if len(sample_words) < 5:
    sample_words = WORD_LIST[:8]

guess = "CRANE"
answer = sample_words[0] if sample_words else "FLOUR"
pattern = get_wordle_pattern(guess, answer)

print(f"Sample from {len(WORD_LIST)} total words: {sample_words}")
print(f"After guessing '{guess}' with answer '{answer}':")
print(f"Pattern: {pattern}")
remaining = [w for w in sample_words if apply_constraints(w, guess, pattern)]
print(f"Remaining possibilities: {remaining}")
print(f"Eliminated: {set(sample_words) - set(remaining)}")
print(f"\nFrom full word list, {len([w for w in WORD_LIST if apply_constraints(w, guess, pattern)])} words remain")

=== Loading Word List ===

Loaded 9972 five-letter words from NLTK
Sample words: ['AALII', 'AARON', 'ABACA', 'ABACK', 'ABAFF', 'ABAFT', 'ABAMA', 'ABASE', 'ABASH', 'ABASK']

=== Wordle Feedback Mechanism ===

Guess: ROBOT
Answer: FLOOR
Pattern: 🟨🟨⬜🟩⬜ (1, 1, 0, 2, 0)

Guess: CRANE
Answer: REACT
Pattern: 🟨🟨🟩⬜🟨 (1, 1, 2, 0, 1)

Guess: SALET
Answer: LASER
Pattern: 🟨🟩🟨🟩⬜ (1, 2, 1, 2, 0)

=== Constraint Propagation Example ===

Sample from 9972 total words: ['FLACK', 'FLAFF', 'FLAIL', 'FLAIR', 'FLAKE', 'FLAKY', 'FLAMB', 'FLAME']
After guessing 'CRANE' with answer 'FLACK':
Pattern: (1, 0, 2, 0, 0)
Remaining possibilities: ['FLACK']
Eliminated: {'FLAIL', 'FLAKY', 'FLAFF', 'FLAME', 'FLAMB', 'FLAIR', 'FLAKE'}

From full word list, 49 words remain


In [2]:
from collections import defaultdict, Counter

# Letter frequency data from Wikipedia (English text)
# https://en.wikipedia.org/wiki/Letter_frequency
LETTER_FREQUENCIES = {
    'E': 12.70, 'T': 9.06, 'A': 8.17, 'O': 7.51, 'I': 6.97,
    'N': 6.75, 'S': 6.33, 'H': 6.09, 'R': 5.99, 'D': 4.25,
    'L': 4.03, 'C': 2.78, 'U': 2.76, 'M': 2.41, 'W': 2.36,
    'F': 2.23, 'G': 2.02, 'Y': 1.97, 'P': 1.93, 'B': 1.29,
    'V': 0.98, 'K': 0.77, 'J': 0.15, 'X': 0.15, 'Q': 0.10,
    'Z': 0.07
}

def build_frequency_tables(words):
    """Calculate letter frequencies by position from word list"""
    position_freq = [defaultdict(int) for _ in range(5)]
    
    for word in words:
        for i, letter in enumerate(word):
            position_freq[i][letter] += 1
    
    return position_freq

def score_word_frequency(word, freq_tables, remaining_words):
    """Score word based on letter/position frequencies"""
    # Use position-specific frequencies from the remaining words
    score = sum(freq_tables[i][letter] for i, letter in enumerate(word))
    
    # Bonus for unique letters (more information gathered)
    unique_ratio = len(set(word)) / len(word)
    score *= (1 + unique_ratio)
    
    # Add bonus based on general English letter frequency
    english_freq_score = sum(LETTER_FREQUENCIES.get(letter, 0) for letter in set(word))
    score += english_freq_score * 0.1  # Small weight for general frequency
    
    return score

def frequency_agent_solve(answer, word_list, verbose=True):
    """Solve Wordle using frequency heuristics"""
    remaining = word_list.copy()
    guesses = []
    
    for attempt in range(6):
        freq_tables = build_frequency_tables(remaining)
        
        # Choose word with highest frequency score from remaining possibilities
        best_word = max(remaining, key=lambda w: score_word_frequency(w, freq_tables, remaining))
        guesses.append(best_word)
        
        pattern = get_wordle_pattern(best_word, answer)
        
        if verbose:
            pattern_str = ''.join(['⬜' if p == 0 else '🟨' if p == 1 else '🟩' for p in pattern])
            print(f"Attempt {attempt + 1}: {best_word} -> {pattern_str}")
            print(f"  Remaining words: {len(remaining)}")
        
        if best_word == answer:
            if verbose:
                print(f"✓ Solved in {len(guesses)} guesses!\n")
            return guesses
        
        # Update remaining words based on constraints
        remaining = [w for w in remaining if apply_constraints(w, best_word, pattern)]
        
        if not remaining:
            if verbose:
                print(f"✗ No valid words remain! Answer was {answer}\n")
            return guesses
    
    if verbose:
        print(f"✗ Failed to solve in 6 guesses. Answer was {answer}\n")
    return guesses

# Test the frequency agent
print("=== Frequency-Based Heuristic Agent ===\n")
print(f"Total word list size: {len(WORD_LIST)}\n")

# Select diverse test words
import random
random.seed(42)
test_answers = random.sample(WORD_LIST, 3)

print(f"Testing on random words: {test_answers}\n")

for answer in test_answers:
    print(f"Target word: {answer}")
    result = frequency_agent_solve(answer, WORD_LIST)
    print()

# Show top words by frequency
print("=== Analysis: Best Opening Words by Frequency ===\n")
freq_tables = build_frequency_tables(WORD_LIST)
all_scores = [(w, score_word_frequency(w, freq_tables, WORD_LIST)) for w in WORD_LIST]
all_scores.sort(key=lambda x: x[1], reverse=True)

print("Top 10 words by frequency score:")
for i, (word, score) in enumerate(all_scores[:10], 1):
    unique_letters = len(set(word))
    print(f"{i:2}. {word}: {score:.2f} ({unique_letters} unique letters)")

=== Frequency-Based Heuristic Agent ===

Total word list size: 9972

Testing on random words: ['CIDER', 'AMPLY', 'KETCH']

Target word: CIDER
Attempt 1: SAITE -> ⬜⬜🟨⬜🟨
  Remaining words: 9972
Attempt 2: FINER -> ⬜🟩⬜🟩🟩
  Remaining words: 259
Attempt 3: HIVER -> ⬜🟩⬜🟩🟩
  Remaining words: 39
Attempt 4: MIDER -> ⬜🟩🟩🟩🟩
  Remaining words: 26
Attempt 5: CIDER -> 🟩🟩🟩🟩🟩
  Remaining words: 4
✓ Solved in 5 guesses!


Target word: AMPLY
Attempt 1: SAITE -> ⬜🟨⬜⬜⬜
  Remaining words: 9972
Attempt 2: CORAL -> ⬜⬜⬜🟨🟨
  Remaining words: 822
Attempt 3: PLAGA -> 🟨🟨🟨⬜⬜
  Remaining words: 68
Attempt 4: AMPLY -> 🟩🟩🟩🟩🟩
  Remaining words: 2
✓ Solved in 4 guesses!


Target word: KETCH
Attempt 1: SAITE -> ⬜⬜⬜🟨🟨
  Remaining words: 9972
Attempt 2: TENET -> 🟨🟩⬜⬜⬜
  Remaining words: 226
Attempt 3: KETCH -> 🟩🟩🟩🟩🟩
  Remaining words: 15
✓ Solved in 3 guesses!


=== Analysis: Best Opening Words by Frequency ===

Top 10 words by frequency score:
 1. SAITE: 11550.32 (5 unique letters)
 2. BARIE: 11485.51 (5 unique letters)


In [3]:
import math

def calculate_pattern_entropy(word, possible_words):
    """
    Calculate expected information gain (entropy) for a guess.
    Higher entropy = more expected information gained.
    """
    if not possible_words:
        return 0
    
    # Count how many words produce each pattern
    pattern_counts = {}
    for candidate in possible_words:
        pattern = get_wordle_pattern(word, candidate)
        pattern_counts[pattern] = pattern_counts.get(pattern, 0) + 1
    
    # Calculate entropy: -sum(p * log2(p))
    total = len(possible_words)
    entropy = 0
    for count in pattern_counts.values():
        if count > 0:
            probability = count / total
            entropy -= probability * math.log2(probability)
    
    return entropy

def entropy_agent_solve(answer, word_list, verbose=True):
    """Solve Wordle using information theory (entropy maximization)"""
    remaining = word_list.copy()
    guesses = []
    
    for attempt in range(6):
        # For first guess, we can pre-compute on full list
        # For subsequent guesses, only consider remaining words for efficiency
        if attempt == 0 and len(remaining) > 1000:
            # Sample for efficiency on first guess with large word list
            sample_size = min(2000, len(remaining))
            candidates = random.sample(remaining, sample_size)
        else:
            candidates = remaining
        
        # Calculate entropy for candidate guesses
        word_scores = [(w, calculate_pattern_entropy(w, remaining)) for w in candidates]
        word_scores.sort(key=lambda x: x[1], reverse=True)
        
        best_word = word_scores[0][0]
        best_entropy = word_scores[0][1]
        guesses.append(best_word)
        
        pattern = get_wordle_pattern(best_word, answer)
        
        if verbose:
            pattern_str = ''.join(['⬜' if p == 0 else '🟨' if p == 1 else '🟩' for p in pattern])
            print(f"Attempt {attempt + 1}: {best_word} -> {pattern_str}")
            print(f"  Entropy: {best_entropy:.2f} bits")
            print(f"  Remaining words: {len(remaining)}")
        
        if best_word == answer:
            if verbose:
                print(f"✓ Solved in {len(guesses)} guesses!\n")
            return guesses
        
        remaining = [w for w in remaining if apply_constraints(w, best_word, pattern)]
        
        if not remaining:
            if verbose:
                print(f"✗ No valid words remain! Answer was {answer}\n")
            return guesses
    
    if verbose:
        print(f"✗ Failed to solve in 6 guesses. Answer was {answer}\n")
    return guesses

# Test the entropy agent
print("=== Information Theory (Entropy) Agent ===\n")

# Use same test words for comparison
random.seed(42)
test_answers = random.sample(WORD_LIST, 3)

print(f"Testing on: {test_answers}\n")

for answer in test_answers:
    print(f"Target word: {answer}")
    result = entropy_agent_solve(answer, WORD_LIST)
    print()

# Compare first guesses
print("=== First Guess Comparison ===\n")

# Sample words for entropy calculation (full list too slow)
print("Calculating best opening words (sampling for efficiency)...\n")
sample_words = random.sample(WORD_LIST, min(1000, len(WORD_LIST)))

print("Top 5 words by frequency score:")
freq_tables = build_frequency_tables(WORD_LIST)
freq_scores = [(w, score_word_frequency(w, freq_tables, WORD_LIST)) for w in sample_words]
freq_scores.sort(key=lambda x: x[1], reverse=True)
for word, score in freq_scores[:5]:
    print(f"  {word}: {score:.2f}")

print("\nTop 5 words by entropy:")
entropy_scores = [(w, calculate_pattern_entropy(w, WORD_LIST)) for w in sample_words]
entropy_scores.sort(key=lambda x: x[1], reverse=True)
for word, entropy in entropy_scores[:5]:
    print(f"  {word}: {entropy:.2f} bits")

=== Information Theory (Entropy) Agent ===

Testing on: ['CIDER', 'AMPLY', 'KETCH']

Target word: CIDER
Attempt 1: SERAI -> ⬜🟨🟨⬜🟨
  Entropy: 5.89 bits
  Remaining words: 9972
Attempt 2: DITER -> 🟨🟩⬜🟩🟩
  Entropy: 3.74 bits
  Remaining words: 165
Attempt 3: BIDER -> ⬜🟩🟩🟩🟩
  Entropy: 0.65 bits
  Remaining words: 6
Attempt 4: CIDER -> 🟩🟩🟩🟩🟩
  Entropy: 0.72 bits
  Remaining words: 5
✓ Solved in 4 guesses!


Target word: AMPLY
Attempt 1: TERAS -> ⬜⬜⬜🟨⬜
  Entropy: 5.83 bits
  Remaining words: 9972
Attempt 2: MANIA -> 🟨🟨⬜⬜⬜
  Entropy: 5.45 bits
  Remaining words: 950
Attempt 3: ALBUM -> 🟩🟨⬜⬜🟨
  Entropy: 3.88 bits
  Remaining words: 23
Attempt 4: AMPLY -> 🟩🟩🟩🟩🟩
  Entropy: 1.00 bits
  Remaining words: 2
✓ Solved in 4 guesses!


Target word: KETCH
Attempt 1: SINAE -> ⬜⬜⬜⬜🟨
  Entropy: 5.86 bits
  Remaining words: 9972
Attempt 2: RELET -> ⬜🟩⬜⬜🟨
  Entropy: 5.63 bits
  Remaining words: 699
Attempt 3: HETTY -> 🟨🟩🟩⬜⬜
  Entropy: 3.35 bits
  Remaining words: 20
Attempt 4: FETCH -> ⬜🟩🟩🟩🟩
  Entropy: 0.92 b